In [1]:
with open('./s2orc_full_text_nlp_ml.txt') as f:
    lines = f.read().splitlines()

In [2]:
# from urllib.request import urlretrieve
# import gzip, shutil

# for fname in lines:
#     if not fname.startswith('https:/'):
#         fname = 'https://ai2-s2-research-public.s3-us-west-2.amazonaws.com/lucas/s2orc' + fname.split('s2orc')[1]
        
#     path, _ = urlretrieve(fname, './raw-data/' + fname.split('/')[-1])

#     with gzip.open(path, 'r') as f_in, open('.'.join(path.split('.')[:-1]).replace('raw-data', 'processed-data'), 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [3]:
from urllib.request import urlretrieve
import pandas as pd
import os

path = "raw-data"
does_exist = os.path.exists(path)
if not does_exist:
    os.makedirs(path)

    for fname in lines:
        if not fname.startswith('https:/'):
            fname = 'https://ai2-s2-research-public.s3-us-west-2.amazonaws.com/lucas/s2orc' + fname.split('s2orc')[1]

        path, _ = urlretrieve(fname, './raw-data/' + fname.split('/')[-1])

        pd.read_parquet(path)
        print(path)

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import pos_tag

[nltk_data] Downloading package wordnet to /Users/arjuns/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/arjuns/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/arjuns/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/arjuns/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
def process_text(text):
    wnpos = lambda e: ('a' if e[0].lower() == 'j' else e[0].lower()) if e[0].lower() in ['n', 'r', 'v'] else 'n'
    wnl = WordNetLemmatizer()

    tokens = word_tokenize(text)
    tokens_pos = pos_tag(tokens)
    lemmas = []
    for t, pos in tokens_pos:
        new_pos = wnpos(pos)
        lemmas.append(wnl.lemmatize(t, pos=new_pos))
    return lemmas

In [6]:
import glob

search_terms = ['democra']

filtered_papers = set()
matches = set()
num_nonreadable = 0

for filepath in glob.iglob('raw-data/*'):
    df = pd.read_parquet(filepath)
    
    for index, row in df.iterrows():
        
        for target in [row['metadata']['title'], row['metadata']['abstract'], row['content']['grobid']['contents']]:
            if target:
                target = target.lower()
            else:
                num_nonreadable += 1
                continue
            
            for s in search_terms:
                if s in target.lower():
                    target = target.lower()
                    filtered_papers.add(row['id'])
                    matches.update([m for m in target.split('.') if s in m])

success


In [7]:
num_nonreadable

2082

In [8]:
matches

{'len \n\n21: \n\n23: \n\n24: \n\nend for \n\n25: \n\nreturn ranks \n26: end procedure \n\n\n\ntable 1 :\n1focused summarization dataset statisticsparty \nmethod \nrouge-1 rouge-2 rouge-l \n\ndemocrat \ntextrank \n17',
 " reference about half of democratic primary voters nationwide say they would enthusiastically support clinton if she became the party's nominee",
 ' this probably explains the high impact of "sponsor is a democrat" (table 2); democrats led the 111th congress, and introduced more bills, most of which died',
 ' hyperbolicity measures democracy in real-world networks',
 ' 10 downing street, the official residence of british prime minister gordon brown, calling on the british government to demand that full democracy be restored in pakistan',
 ' democratic regeneration encompasses concepts such as calls for constitutional amendments or changes, favourable mentions of the system of direct democracy, the need of involvement of all citizens in political decision-making, divisi

In [9]:
filtered_papers

{260063238,
 255393799,
 235266057,
 254246921,
 259833868,
 250150925,
 7577640,
 246276138,
 852013,
 249282609,
 219308082,
 14082107,
 3145799,
 209387593,
 104292427,
 209387602,
 8953944,
 247447641,
 247595108,
 199557221,
 219299945,
 237420654,
 248512629,
 251224184,
 247595140,
 252846223,
 219308180,
 219308188,
 6906014,
 17146014,
 7094438,
 248512698,
 3137722,
 12419266,
 237420740,
 2400465,
 1851612,
 118685920,
 225067238,
 208191718,
 253116650,
 2998513,
 253370611,
 219635956,
 219300084,
 197640450,
 237502733,
 246702350,
 11878680,
 21709085,
 51872030,
 237592862,
 247595303,
 21700910,
 221266228,
 6578489,
 11469122,
 14647620,
 232333645,
 21700944,
 245334353,
 219308373,
 253116763,
 2195807,
 7692647,
 261341546,
 27378030,
 14565746,
 311673,
 219300220,
 2736514,
 1114511,
 1384861,
 219636132,
 237945261,
 15827373,
 311736,
 53576131,
 91177412,
 252846537,
 219308497,
 786898,
 247628243,
 252781014,
 11821532,
 248299996,
 233210350,
 253018612,
 2